Some useful links from Dana:

https://scikit-image.org/docs/stable/auto_examples/edges/plot_skeleton.html

https://nalinc.github.io/blog/2018/skin-detection-python-opencv/

https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html

The paper and video we're basing everythign off of:

https://link.springer.com/article/10.1007/s11042-013-1501-1

https://www.youtube.com/watch?v=xML2S6bvMwI

In [5]:
import cv2
import numpy as np
import math

# bullet = cv2.imread("470-FingerGunGame/images/small_bullet3.png")

def superimpose(img_1, img_2):
    img_1_gs = cv2.cvtColor(img_1, cv2.COLOR_BGR2GRAY)
    _ret, mask = cv2.threshold(img_1_gs, 10, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)
    roi = cv2.bitwise_and(img_1, img_1, mask=mask)
    im2 = cv2.bitwise_and(img_2, img_2, mask=mask_inv)
    result = cv2.add(im2, roi)
    return result

# top_left is the point where the top left of overlay_img should line up
# top_right is the point where the top right of overlay_img should line up
# both are in the background_img coordinate space
def rotate_overlay_onto_line(overlay_img, background_img, top_left, top_right, top_offset = 0, left_offset = 0, right_offset = 0, flipped=False):

    overlay_h, overlay_w = overlay_img.shape[:2]
    background_h, background_w = background_img.shape[:2]

    # ratio to scale up the overlay img by
    ratio = math.dist(top_left, top_right) / overlay_w
    
    # points needed for the affine transformation
    # in order: top left, top right, bottom right
    # needs to be x, y

    # the offset will scale a bit in the bottom right depending the the width to keep proportions
    offset_scale = (overlay_w + right_offset) / overlay_w
    pts1 = np.float32([[0-left_offset, 0-top_offset], [overlay_w-right_offset, 0-top_offset], [overlay_w-right_offset, (overlay_h/offset_scale)-top_offset]])

    # used to determine where the bottom right point should be
    line_angle = np.arctan2(top_right[1] - top_left[1], top_right[0] - top_left[0])

    x_offset = math.sin(line_angle) * overlay_h * ratio
    y_offset = math.cos(line_angle) * overlay_h * ratio

    if flipped:
        bottom_right = [int(top_right[0] + x_offset), int(top_right[1] - y_offset)]
    else:
        bottom_right = [int(top_right[0] - x_offset), int(top_right[1] + y_offset)]

    pts2 = np.float32([top_left, top_right, bottom_right])

    # get the affine transformation matrix
    M = cv2.getAffineTransform(pts1, pts2)

    # apply the affine transform to the overlay image
    warped_overlay = cv2.warpAffine(overlay_img, M, (background_w, background_h))

    # superimpose onto the background
    return superimpose(warped_overlay, background_img)


def draw_target(img, center, radius, color, alt_color=[255, 255, 255], num_circles=3):
    for circle in range(num_circles):

        if circle % 2:
            target_color = alt_color
        else:
            target_color = color

        radius_multiplier = 1 - circle/num_circles

        cv2.circle(img, center, int(radius_multiplier * radius), target_color, -1)


# get the angle between ab and ac given a, b, c xy-points of a triangle
# uses the law of cosines aka the cosine rule / formula
def cos_rule(a, b, c):
    # euclidian distance between points
    ab = math.dist(a, b)
    bc = math.dist(b, c)
    ac = math.dist(a, c)

    # using law of cosines to compute angle between ab and bc

    angle_abac = math.degrees(math.acos((ab ** 2 + ac ** 2 - bc ** 2) / (2 * ab * ac)))
    return angle_abac


In [6]:
import numpy as np
import math
import cv2
import random
import time


cap = cv2.VideoCapture(1)

background = cv2.imread("images/image1.png")
gun = cv2.imread("images/revolver2.png")
# bullet = cv2.imread("small_bullet.png", cv2.IMREAD_UNCHANGED)

# skin colour numbers
# y_min = 25 #64
# y_max = 193
# cr_min = 139 # 133
# cr_max = 173
# cb_min = 77
# cb_max = 127

y_min = 23
y_max = 244
cr_min = 145
cr_max = 173
cb_min = 77
cb_max = 127


debug_level = 0
# 0 = nothing extra drawn
# 1 = contours
# 2 = hull
# 3 = hull defects
# 4 = thumb area
# 5 = palm circle and biggest circle
# 6 = pistol

def fingerGunGame(debug_level=0):
    # colors (B, G, R)
    bullet_color = (0, 0, 0)
    bullet_radius = 9

    gun_flipped = False
    gun_topl = None
    gun_topr = None
    fire_ready = False
    bullets = []
    next_bullet = None

    shots_fired = 0
    targets_hit = 0

    bullet_speed = 9
    exit_angle = None
    start_game = False
    gameFinished = False
    score = 0

    win_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    win_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    # resize the background overlay
    background_scaled = cv2.resize(background, [win_w, win_h], interpolation=cv2.INTER_AREA)

    centre1 = [random.randint(100,1000), random.randint(200,350)]
    centre2 = [random.randint(100,1000), random.randint(450,700)]
    centre3 = [random.randint(100,1000), random.randint(800,980)]
    radius1 = random.randint(30,100)
    radius2 = random.randint(30,100)
    radius3 = random.randint(30,100)
    start_time = 0

    # how long the game should last in seconds
    game_length = 20


    while start_time >= 0:
        _ret, img = cap.read()
        drawing = np.zeros(img.shape, np.uint8)
        #grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kernel_size = (35, 35)
        
        img_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        #blurred = cv2.GaussianBlur(grey, kernel_size, 0)
        blurred = cv2.GaussianBlur(img_ycrcb, kernel_size, 0)

        scale_factor = 4
        if scale_factor != 1:
            blurred = cv2.resize(blurred, [blurred.shape[1]//scale_factor, blurred.shape[0]//scale_factor], interpolation=cv2.INTER_AREA)
        
        # skin color segmentation
        skin_ycrcb_mint = np.array([y_min, cr_min, cb_min], np.uint8)
        skin_ycrcb_maxt = np.array([y_max, cr_max, cb_max], np.uint8)
        thresholded = cv2.inRange(blurred, skin_ycrcb_mint, skin_ycrcb_maxt)
        #_ret, thresholded = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        contours, _hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #contours, _hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        # mix the background with the image
        if (debug_level == 0):
            img = cv2.addWeighted(img, 0.2, background_scaled, 0.8, 0)

        # if no contours are detected, don't draw anything
        if len(contours) != 0:
            # this is the largest contour found, which helps eliminate noise and should be the hand
            count1 = max(contours, key=lambda x: cv2.contourArea(x))

            # drawing the biggest enclosing circle of the contour shape, used for gesture recognition
            big_c, big_r = cv2.minEnclosingCircle(count1)
            big_c = (int(big_c[0])*scale_factor, int(big_c[1])*scale_factor)

            #if ((big_r*scale_factor) > (3.5* center_dist*scale_factor)):

            if debug_level >= 5:
                # center point
                cv2.circle(img, big_c, 1, (255, 0, 0), -1)
                # circle containing hand
                cv2.circle(img, big_c, int(big_r)*scale_factor, (255, 0, 0), 3)


            # center point
            cv2.circle(drawing, big_c, 1, (255, 0, 0), -1)
            # circle containing hand
            cv2.circle(drawing, big_c, int(big_r)*scale_factor, (255, 0, 0), 3)
            
            x, y, w, h = cv2.boundingRect(count1)

            # finding the palm center
            center_pt = (0, 0)
            center_dist = 0
            for pt_x in range(x, x+w, 2):
                for pt_y in range(y, y+h, 2):
                    pt = (pt_x, pt_y)
                    dist = cv2.pointPolygonTest(count1, pt, measureDist=True)
                    if dist > center_dist:
                        center_pt = (pt_x * scale_factor, pt_y * scale_factor)
                        center_dist = dist
                    #cv2.circle(img, pt, 1, (255, 0, 255), 1)
                    #else:
                    #    cv2.circle(img, pt, 1, (0, 0, 0), 1)
            
            # biggest circle in palm
            if debug_level >= 5:
                cv2.circle(img, center_pt, int(center_dist)*scale_factor, (255, 255, 0), 3)
                # circle 3.5 * the radius
                #cv2.circle(img, center_pt, int(center_dist*3.5)*scale_factor, (255, 255, 0), 2)
                # biggest circle / 2
                #cv2.circle(img, center_pt, int(center_dist//2)*scale_factor, (0, 165, 255), 2)
                # center of biggest circle
                cv2.circle(img, center_pt, 1, (255, 255, 0), -1)
            
            cv2.circle(drawing, center_pt, int(center_dist)*scale_factor, (255, 255, 0), 3)
            # circle 3.5 * the radius
            #cv2.circle(img, center_pt, int(center_dist*3.5)*scale_factor, (255, 255, 0), 2)
            # biggest circle / 2
            #cv2.circle(drawing, center_pt, int(center_dist//2)*scale_factor, (0, 165, 255), 2)
            # center of biggest circle
            cv2.circle(drawing, center_pt, 1, (255, 255, 0), -1)

            # not necessary at the moment
            # rotRect = cv2.minAreaRect(count1)
            # box = cv2.boxPoints(rotRect)
            # box = np.int64(box)
            # cv2.drawContours(img, [box*scale_factor], 0, (255, 255, 0), 1)

            #cv2.rectangle(img, (x*scale_factor, y*scale_factor), ((x + w)*scale_factor, (y + h)*scale_factor), (0, 0, 255), 1)

            #hull = cv2.convexHull(count1)
            try:
                hull = cv2.convexHull(count1, returnPoints=False)
                defects = cv2.convexityDefects(count1, hull)
            except:
                # print(hull, count1)
                continue
                raise IOError("Hull issue")
            
            # drawing contours
            if debug_level >= 1:
                cv2.drawContours(img, [count1*scale_factor], 0, (0, 255, 0), 0)


            cv2.drawContours(drawing, [count1*scale_factor], 0, (0, 255, 0), 0)
            #cv2.drawContours(drawing, [hull*scale_factor], 0, (0, 0, 255), 0)
            count_defects = 0
            #cv2.drawContours(thresholded, contours*scale_factor, -1, (0, 255, 0), 3)
            

            # doing contour extractions once again within the 3.5x circle
            # grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # blurred2 = cv2.GaussianBlur(grey, kernel_size, 0)
            # _ret, thresholded2 = cv2.threshold(blurred2, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            # contours2, _hierarchy = cv2.findContours(thresholded2.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#cv2.CHAIN_APPROX_NONE)

            # if len(contours2) != 0:
            #     count1_2 = max(contours2, key=lambda x: cv2.contourArea(x))


            # print(len(contours), defects.shape, '\n\n', contours)
            # break

            # go through all the defects in the hull
            if defects is not None:
                finger_tips = []
                finger_valleys = []
                angles = []
                thumb_defect = None
                biggest_angle = 0
                for defect in defects:
                    # start index, end index, farthest pt index, depth
                    s, e, f, d = defect[0]
                    start = tuple(count1[s][0])
                    end = tuple(count1[e][0])
                    far = tuple(count1[f][0])


                    if debug_level >= 2:
                        # draw a line from the start to the end to show the hull
                        cv2.line(img, (start[0]*scale_factor, start[1]*scale_factor), (end[0]*scale_factor, end[1]*scale_factor), [0, 0, 255], 2)
                    
                    cv2.line(drawing, (start[0]*scale_factor, start[1]*scale_factor), (end[0]*scale_factor, end[1]*scale_factor), [0, 0, 255], 2)

                    # depth has 8 fractional bits, to get the value as a float divide by 256
                    depth = d // 256

                    angle = cos_rule(far, start, end)

                    # cosidered a finger when the angle is < 90º and depth of defect is between the small and large radius
                    if angle <= 100 and depth > center_dist and depth < big_r:

                        count_defects += 1
                        finger_tips.append([s, e, f, d])
                        angles.append(angle)
                        max(contours, key=lambda x: cv2.contourArea(x))

                        if angle > biggest_angle:
                            biggest_angle = angle
                            thumb_defect = [start, end, far]

                        if debug_level >= 3:
                            # draw circles at each of the start, far, and end points
                            cv2.circle(img, (start[0]*scale_factor, start[1]*scale_factor), 3, [255, 0, 255], 3)
                            cv2.circle(img, (end[0]*scale_factor, end[1]*scale_factor), 3, [255, 0, 255], 3)
                            cv2.circle(img, (far[0]*scale_factor, far[1]*scale_factor), 3, [255, 0, 255], 3)
                        
                        cv2.circle(drawing, (start[0]*scale_factor, start[1]*scale_factor), 3, [255, 0, 255], 3)
                        cv2.circle(drawing, (end[0]*scale_factor, end[1]*scale_factor), 3, [255, 0, 255], 3)
                        cv2.circle(drawing, (far[0]*scale_factor, far[1]*scale_factor), 3, [255, 0, 255], 3)
                        #print(f"defect {count_defects}: start: {start}, end: {end}, far: {far}, ra: {int(center_dist)}, l: {depth}, rb: {int(big_r)}")

                # for a in range(len(angles)):
                biggest_area = 0
                thumb = None

                for finger_tip in finger_tips:
                    # start, end, far indices of the contour
                    s, e, f, _d = finger_tip

                    # check that the far point y is not above the thumb tip (aka it accidentally thinks the pinkie knuckle is the thumb tip)
                    start = count1[s][0]
                    end = count1[e][0]
                    far = count1[f][0]

                    tip_1 = math.dist(start, far)
                    tip_2 = math.dist(end, far)

                    if (tip_1 > tip_2):
                        thumb_tip = end
                    else:
                        thumb_tip = start
                    
                    area = cv2.contourArea(count1[s:e+1])
                    if thumb_tip[1] < far[1] and area > biggest_area:
                        biggest_area = area
                        thumb = finger_tip
                    #biggest_area = max(finger_tips, key=lambda x: cv2.contourArea(count1[x[0]:x[1]+1]))
                
                if thumb is not None and start_game:
                    s, e, f, _d = thumb
                    count1_reshaped = count1.reshape(-1, 2)

                    if debug_level >= 4:
                        # filling the area between the thumb and pointer finger with blue to visually confirm the gun is working
                        cv2.fillPoly(img, [count1_reshaped[s:e+1]*scale_factor], (255, 0, 0))

                    cv2.fillPoly(drawing, [count1_reshaped[s:e+1]*scale_factor], (255, 0, 0))
                    
                    start = count1[s][0]
                    end = count1[e][0]
                    far = count1[f][0]
                    angle = cos_rule(far, start, end)

                    tip_1 = math.dist(start, far)
                    tip_2 = math.dist(end, far)

                    if (tip_1 > tip_2):
                        pointer_tip = start
                        thumb_tip = end
                    else:
                        pointer_tip = end
                        thumb_tip = start

                    # the line along the top of the pointer finger
                    if (debug_level >= 3):
                        cv2.line(img, pointer_tip*scale_factor, far*scale_factor, (255, 0, 255), 2)
                        cv2.line(img, thumb_tip*scale_factor, far*scale_factor, (255, 0, 255), 2)

                    cv2.line(drawing, pointer_tip*scale_factor, far*scale_factor, (255, 0, 255), 2)
                    cv2.line(drawing, thumb_tip*scale_factor, far*scale_factor, (255, 0, 255), 2)

                    gun_topl = pointer_tip*scale_factor
                    gun_topr = far*scale_factor

                    # set the flipped flag
                    if gun_topl[0] > gun_topr[0]:
                        gun_flipped = True
                    else:
                        gun_flipped = False

                    # if angle < 80:
                    #     fire_ready = False
                    # else:
                    #cv2.putText(img, f"Angle: {angle:3.0f} - aim", (1000, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2)
                    fire_ready = True
                    next_bullet = pointer_tip

                    x_dist = pointer_tip[0] - far[0]
                    y_dist = pointer_tip[1] - far[1]
                    exit_angle = math.atan2(y_dist, x_dist)

            if fire_ready and count_defects == 0:
                # cv2.putText(img, f"Fire!", (1000, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4)
                fire_ready = False

                if next_bullet is not None:
                    shots_fired += 1
                    bullets.append([next_bullet, exit_angle])

                    next_x = next_bullet[0] + math.cos(exit_angle) * bullet_speed
                    next_y = next_bullet[1] + math.sin(exit_angle) * bullet_speed
                    next_coords = (next_x, next_y)
                    #print(f"bullet_coords: {next_bullet}, angle: {math.degrees(exit_angle)}, next_coords: {next_coords}")

            #if count_defects < 5:
            #    cv2.putText(img, f"{count_defects+1} finger(s)", (50, win_h - 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4)
                
            if not start_game and count_defects == 4:
                start_game = True
                start_time = time.time()

        new_bullets = []

        # draw the targets
        draw_target(img, centre1, radius1, [255, 0, 0])
        draw_target(img, centre2, radius2, [0, 255, 0])
        draw_target(img, centre3, radius3, [0, 0, 255])

        if gameFinished == False:
            for b in range(len(bullets)):

                bullet_coord, bullet_angle = bullets[b]
                #print(bullet_coord)
                # target 1 collision detection
                if (math.dist(bullet_coord*scale_factor, centre1) <= radius1 + bullet_radius):
                    targets_hit += 1
                    # print("HIT")
                    if 30<=radius1<50:
                        score+= 15
                        # cv2.putText(img, f"+15", centre1, cv2.FONT_HERSHEY_SIMPLEX, 2, (19, 41, 1), 4)
                        # cv2.waitKey(200)
                    elif 50<=radius1<80:
                        score+= 10
                        # cv2.putText(img, f"+10", centre1, cv2.FONT_HERSHEY_SIMPLEX, 2, (19, 41, 1), 4)
                        # cv2.waitKey(200)
                    elif 80<= radius1 < 100:
                        score+=5
                        # cv2.putText(img, f"+5", centre1, cv2.FONT_HERSHEY_SIMPLEX, 2, (19, 41, 1), 4)
                        # cv2.waitKey(200)
                    radius1 = random.randint(30,100)
                    centre1 = [random.randint(100,1000), random.randint(200,350)]
                    continue

            # target 2 collision detection
                elif (math.dist(bullet_coord*scale_factor, centre2) <= radius2 + bullet_radius):
                    targets_hit += 1
                    # print("HIT")
                    if 30<=radius2<50:
                        score+= 15
                        # cv2.putText(img, f"+15", centre2, cv2.FONT_HERSHEY_SIMPLEX, 2, (19, 41, 1), 4)
                        # cv2.waitKey(200)
                    elif 50<=radius2<80:
                        score+= 10
                        # cv2.putText(img, f"+10", centre2, cv2.FONT_HERSHEY_SIMPLEX, 2, (19, 41, 1), 4)
                        # cv2.waitKey(200)
                    elif 80<= radius2 < 100:
                        score+=5
                        # cv2.putText(img, f"+5", centre2, cv2.FONT_HERSHEY_SIMPLEX, 2, (19, 41, 1), 4)
                        # cv2.waitKey(200)
                    radius2 = random.randint(30,100)
                    centre2 = [random.randint(100,1000), random.randint(450,700)]
                    continue

                # target 3 collision detection
                elif (math.dist(bullet_coord*scale_factor, centre3) <= radius3 + bullet_radius):
                    targets_hit += 1
                    # print("HIT")
                    if 30<=radius3<50:
                        score+= 15
                        # cv2.putText(img, f"+15", centre3, cv2.FONT_HERSHEY_SIMPLEX, 2, (19, 41, 1), 4)
                        # cv2.waitKey(200)
                    elif 50<=radius3<80:
                        score+= 10
                        # cv2.putText(img, f"+10", centre3, cv2.FONT_HERSHEY_SIMPLEX, 2, (19, 41, 1), 4)
                        # cv2.waitKey(200)
                    elif 80<= radius3 < 100:
                        score+=5
                        # cv2.putText(img, f"+5", centre3, cv2.FONT_HERSHEY_SIMPLEX, 2, (19, 41, 1), 4)
                        # cv2.waitKey(200)

                    radius3 = random.randint(30,100)
                    centre3 = [random.randint(100,1000), random.randint(800,980)]
                    continue

                # get rid of any bullets not within the screen
                if bullet_coord[0] > 0 and bullet_coord[0] < img.shape[1] and bullet_coord[1] > 0 and bullet_coord[1] < img.shape[0]:
                    cv2.circle(img, bullet_coord*scale_factor, bullet_radius, bullet_color, -1)
                    cv2.circle(drawing, bullet_coord*scale_factor, bullet_radius, [255, 255, 255], -1)

                    # update the bullet location for the next frame
                    bullet_coord[0] += math.cos(bullet_angle) * bullet_speed
                    bullet_coord[1] += math.sin(bullet_angle) * bullet_speed
                    new_bullets.append([bullet_coord, bullet_angle])

        bullets = new_bullets

        
        if start_game:

            if (not gameFinished and gun_topl is not None and (debug_level == 0 or debug_level == 6)):
                img = rotate_overlay_onto_line(gun, img, gun_topl, gun_topr, top_offset=-20, right_offset=60, flipped=gun_flipped)

            elapsed_time = time.time() - start_time
            time_left = game_length - elapsed_time
            if gameFinished or time_left <= 0:
                gameFinished = True
                img = background_scaled
                cv2.putText(img, "Finished!", (830, 400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 6)
                cv2.putText(img, f"Score: {score}", (830, 520), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 6)
                if shots_fired == 0:
                    shots_fired = 1
                accuracy = (targets_hit / shots_fired) * 100
                cv2.putText(img, f"Accuracy: {accuracy:.0f}%", (750, 640), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 6)
                cv2.putText(img, "Press Enter to play again", (780, 800), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 4)
                cv2.putText(img, "Press Esc to quit", (780, 870), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 4)

            elif time_left <= 5:
                cv2.putText(img, f"Time: {time_left:.2f}", (600, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 100), 4)
            else:
                cv2.putText(img, f"Time: {time_left:.2f}", (600, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4)

            if not gameFinished:
                cv2.putText(img, f"Score: {score:4}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4)
                cv2.putText(img, "game started", (win_w - 500, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 100, 0), 4)

        else:
            cv2.putText(img, "waiting", (win_w - 300, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 180), 4)

        all_img = np.vstack((img, drawing))

        cv2.imshow('Finger Guns ', all_img)

        if gameFinished:
            return 0

        k = cv2.waitKey(1)
        if k == 27: # esc
            return 1
        elif k == 2: # left arrow
            debug_level = max(0, debug_level - 1)
        elif k == 3: # right arrow
            debug_level = min(6, debug_level + 1)


while 1:
    retval = fingerGunGame(debug_level)

    if retval:
        break

    k = cv2.waitKey(0)
    if k == 27:
        break

cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release()
